**NOTE**: This is a recording of a previous live coding of the Roomba simulation. It is suggested you start from scratch for your class.

In [ ]:
import math
import random
import matplotlib.pyplot as plt
import statistics as st

In [ ]:
%matplotlib inline

In [ ]:
def angular_motion(degrees, speed=1):
    radians = math.radians(degrees)
    x_motion = math.cos(radians) * speed
    y_motion = math.sin(radians) * speed
    return x_motion, y_motion

In [ ]:
assumed = math.sqrt(0.5)
x, y = angular_motion(45)
print(assumed)
print(x)
print(y)

In [ ]:
angular_motion(135)

In [ ]:
class Room:
    """
    Responsibilities:
    - width
    - height
    - keeps track of what squares are clean
    """
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.clean_squares = set()
        
    def reset(self):
        self.clean_squares = set()
        
    def random_location(self):
        x = random.random() * self.width
        y = random.random() * self.height
        return x, y
    
    def total_squares(self):
        return self.width * self.height
    
    def in_room(self, x, y):
        """Check to see if coordinate is in the room."""
        return 0 <= x < self.width and 0 <= y < self.height
    
    def clean_square(self, x, y):
        if self.in_room(x, y):
            self.clean_squares.add((x, y))
    
    def percent_clean(self):
        return len(self.clean_squares) / self.total_squares()

In [ ]:
room = Room(2, 2)
room.clean_square(0, 0)
print(room.percent_clean())
room.clean_square(0, 0)
print(room.percent_clean())
room.clean_square(0, 1)
print(room.percent_clean())
room.clean_square(5, 4)
print(room.percent_clean())

In [ ]:
class Roomba:
    """
    Responsibilities:
    
    - know its current angle
    - respond to hitting a wall
    - give its relative x and y movement
    - know its speed
    """
    
    def __init__(self, speed=1):
        self.angle = None
        self.speed = speed
        
    def move(self):
        """On trigger, return relative x and y movement."""
        if self.angle is not None:
            return angular_motion(self.angle, self.speed)
        
    def update_angle(self, angle):
        self.angle += angle
        self.angle %= 360
    
    def collide(self):
        """On trigger, choose a new angle."""
        self.angle += random.randint(90, 270)
        self.angle %= 360

In [ ]:
class Simulation:
    """
    Responsibilities:
    - placing the Roomba
    - asking Roomba where it's moving
    - updating the room
    - iterating over turns
    - reporting data
    
    Collaborators:
    - Room
    - Roomba
    """
    
    def __init__(self, room, roomba):
        self.room = room
        self.roomba = roomba
        self.iterations = 0
        
        initial_x, initial_y = self.room.random_location()
        initial_angle = random.randint(0, 359)

        self.current_x = initial_x
        self.current_y = initial_y
        self.roomba.angle = initial_angle
        
    
    def update_roomba_and_check_for_hit(self, x_move, y_move):
        def constrain(min, max, actual):
            if actual < min:
                return min, True
            elif actual > max:
                return max, True
            else:
                return actual, False
        
        tolerance = 0.1
        min_x, min_y = tolerance, tolerance
        max_x, max_y = self.room.width - tolerance, self.room.height - tolerance
        
        x, hit_x = constrain(min_x, max_x, self.current_x + x_move)
        y, hit_y = constrain(min_y, max_y, self.current_y + y_move)
        
        self.current_x = x
        self.current_y = y
        
        return hit_x or hit_y
    
    
    def iterate(self):
        did_hit = self.update_roomba_and_check_for_hit(*self.roomba.move())
        if did_hit:
            self.roomba.collide()
        self.room.clean_square(int(self.current_x), int(self.current_y))
        self.iterations += 1
        
    def report(self):
        return self.iterations, self.room.percent_clean()
        

In [ ]:
room = Room(15, 20)
roomba = Roomba()
simulation = Simulation(room, roomba)

In [ ]:
for _ in range(20):
    simulation.iterate()
    print(simulation.report())

In [ ]:
simulation.room.clean_squares

In [ ]:
room = Room(15, 20)
roomba = Roomba()
simulation = Simulation(room, roomba)

percent_clean = 0
while percent_clean < 0.9:
    simulation.iterate()
    turn, percent_clean = simulation.report()
    
print(turn)

In [ ]:
def run_simulation_by_percentage(room, roomba):
    simulation = Simulation(room, roomba)
    
    percent_clean = 0
    
    while percent_clean < 0.5:
       simulation.iterate()
       turn, percent_clean = simulation.report() 
        
    half_clean = turn
    
    while percent_clean < 0.9:
       simulation.iterate()
       turn, percent_clean = simulation.report() 
        
    almost_clean = turn
    
    while percent_clean < 1.0:
       simulation.iterate()
       turn, percent_clean = simulation.report() 
    
    clean = turn
    
    room.reset()
    
    return half_clean, almost_clean, clean

In [ ]:
run_simulation_by_percentage(Room(15, 20), Roomba())

In [ ]:
def run_simulation_by_percentage_trials(room, roomba, trials=100):
    trial_results = [run_simulation_by_percentage(room, roomba) for _ in range(trials)]
    return trial_results

In [ ]:
trial_results = run_simulation_by_percentage_trials(Room(15, 20), Roomba(), 100)
results_to_plot = list(zip(*trial_results))
plt.boxplot(results_to_plot)
plt.xticks(range(1, 4), ["50%", "90%", "100%"])
# plt.yscale("log")
plt.show()

In [ ]:
results_to_plot = list(zip(*trial_results))
plt.hist(results_to_plot[0], bins=20)
plt.title("Time to 50%")
plt.show()

In [ ]:
plt.hist(results_to_plot[1], bins=20)
plt.title("Time to 90%")
plt.show()

In [ ]:
print(st.mean(results_to_plot[1]))
print(st.pstdev(results_to_plot[1]))

In [ ]:
original = [st.mean(results) for results in results_to_plot]

In [ ]:
plt.plot([50, 90, 100], original, label="Number of turns")
plt.legend()
plt.show()